In [986]:
#put dimension = n, for a n*n puzzle
global dimension 
dimension = 3

In [984]:
import random
#Defining a function to generate the goal state for a given dimension
def goalState():
    size = dimension
    temp = []
    goalState = []
    max = size ** 2
    for x in range (max):
        temp.append(x+1)
        if ((x+1) % size) == 0:
            goalState.append(temp)
            temp = []
    return goalState
        
    
def randomState():
    lis = []
    max = dimension ** 2
    while len(lis) < max:
        r = random.randint(1, int(max))
        if r not in lis:
            lis.append(r)
    
    randomState = []
    
    for x in range(dimension):
        temp = []
        temp = lis[x*dimension:(x+1)*dimension]
        randomState.append(temp)
    
    return randomState
    

In [ ]:
#Representing the goal state for an n-by-n S-Puzzle
goalState = goalState()


In [992]:
#Creating timer necessary for time-limited algorithms
from time import *
import threading

def countdown():
    global my_timer
    my_timer = 60
    
    for x in range(60):
        my_timer = my_timer - 1
        sleep(1)

In [ ]:
#Defining the set of operators generically 
def swapUp(row,col,state):
    if row == 0:
        return state
    else:
        temp = state[row][col]
        state[row][col] = state[row-1][col]
        state[row-1][col] = temp
        return state
        
def swapDown(row,col,state):
    if row == dimension-1:
        return state
    else:
        temp = state[row][col]
        state[row][col] = state[row+1][col]
        state[row+1][col] = temp
        return state
        
def swapLeft(row,col,state):
    if col == 0:
        return state
    else:
        temp = state[row][col]
        state[row][col] = state[row][col-1]
        state[row][col-1] = temp
        return state
    
def swapRight(row,col,state):
    if col == dimension-1:
        return state
    else:
        temp = state[row][col]
        state[row][col] = state[row][col+1]
        state[row][col+1] = temp
        return state
    
    
#Defining the goal-state function
def checkGoal(state):
    if state == goalState:
        return True
    else:
        return False

#Defining a function to generate children 
#Importing copy for deepCopy
import copy 
def getChildren(state):
    children = []
    
    #Computing children resulting from vertical swaps
    for row in range(dimension-1):
        for col in range(dimension):
            tempState = copy.deepcopy(state)
            temp = swapDown(row,col,tempState)
#             print("Printing swapDown at index" + repr(row) + repr(col))
#             print(temp)
            children.append([temp,state])
    
    #Computing children resulting from horizontal swaps
    for row in range(dimension):
        for col in range(dimension-1):
            tempState = copy.deepcopy(state)
            temp = swapRight(row,col,tempState)
#             print("Printing swapRight at index" + repr(row) + repr(col))
#             print(temp)
            children.append([temp,state])
    
    return children


#Defining a function to generate children and their depth
def getChildrenWithDepth(stateDepth):
    children = []
    
    #Computing children resulting from vertical swaps
    for row in range(dimension-1):
        for col in range(dimension):
            tempState = copy.deepcopy(stateDepth[0])
            temp = swapDown(row,col,tempState)
            #node list is of form [state,depth,parent]
            children.append([temp,stateDepth[1]+1,stateDepth[0]])
    
    #Computing children resulting from horizontal swaps
    for row in range(dimension):
        for col in range(dimension-1):
            tempState = copy.deepcopy(stateDepth[0])
            temp = swapRight(row,col,tempState)
#           #node list is of form [state,depth,parent]
            children.append([temp,stateDepth[1]+1,stateDepth[0]])
    
    return children

def getChildrenWithHeuristic(parent,h):
    children = []
    
    #Computing children resulting from vertical swaps
    for row in range(dimension-1):
        for col in range(dimension):
            tempState = copy.deepcopy(parent)
            temp = swapDown(row,col,tempState)

            children.append([temp,parent,h(temp)])
    
    #Computing children resulting from horizontal swaps
    for row in range(dimension):
        for col in range(dimension-1):
            tempState = copy.deepcopy(parent)
            temp = swapRight(row,col,tempState)

            children.append([temp,parent,h(temp)])
    
    return children

In [ ]:
#Defining the depth-first search algorithm

#Defining the open list as a stack implemented through deque
from collections import deque

def depthFirst(initialState):
    success = False
    openStack = deque()
    #Nodes are of the form [state, parent-node]
    openStack.append([initialState,[-1]])
    closed = []
    
    countdown_thread = threading.Thread(target = countdown)
    countdown_thread.start()
    while openStack and (my_timer > 0):
        x = openStack.pop()
        if checkGoal(x[0]):
            closed.append(x)
            success = True
            break
            
        else:
            children = getChildren(x[0])
            closed.append(x)
            for childTuple in children:
                if childTuple in openStack or childTuple in closed:
                    children.remove(childTuple)
                else:
                    #Pushing valid children (left-end of open)
                    openStack.append(childTuple)
        sleep(1)            

    
    #Exited while-loop without finding the goal state - fail
    if success:
        print('SUCCESS')
        solutionPath = []
        searchPath = []
        
        while not x[1] == [-1]:
            solutionPath.append(x[0])
            for node in closed:
                if node[0] == x[1]:
                    x = node
                    break
                    
        solutionPath.append(x[0]) 
        solutionPath.reverse()
        
        #Code to generate searchPath
        while closed:
            searchPath.append(closed.pop()[0])
        searchPath.reverse()
        
        timeTaken = 60-my_timer
        
        return solutionPath, searchPath,timeTaken
       
    else:
        return [-1],-1,-1


In [ ]:
# #Test code
# randomStart = [[1,2,3],[4,5,6],[8,7,9]]
# depthFirst(randomStart)

In [ ]:
#Defining the iterative-deepening search algorithm
def iterativeDeepening(initialState):
    success = False
    openStack = deque()
    #This time the objects in the stack are list of two elements:
    # The state, the depth, and the parent node
    openStack.append([initialState,0,[-1]])
    closed = []
    k = 1
    
    countdown_thread = threading.Thread(target = countdown)
    countdown_thread.start()
    while openStack and (my_timer > 0):
        x = openStack.pop()
        #Loop to apply depth limit
        while x[1] > k:
            if openStack:
                x = openStack.pop()
            else:
                k += 1 #Increment value: Change if needed
                print("K NUMBER INCREMENTED, NOW: " + repr(k))
                openStack.clear()
                closed.clear()
                x = [initialState,0,[-1]]    
        
        #Standard DFS algo
        if checkGoal(x[0]):
            closed.append(x)
            success = True
            break
        else:
            children = getChildrenWithDepth(x)
            closed.append(x)
            #Check if children are already in the open or closed list
            for childTuple in children:
                if childTuple in openStack or childTuple in closed:
                    children.remove(childTuple)
                else:
                    #Pushing valid children (left-end of open)
                    openStack.append(childTuple)
                    
        if not openStack:
            k += 1 #Increment value: Change if needed
            print("K NUMBER INCREMENTED, NOW: " + repr(k))
            openStack.clear()
            closed.clear()
            openStack.append([initialState,0,[-1]])
        sleep(1)     
                    
                                    
  #Exited while-loop without finding the goal state - fail
    if success:
        solutionPath = []
        searchPath = []
        
        #Code to genereate solution path   
        while not x[2] == [-1]:
            solutionPath.append(x[0])
            for node in closed:
                if node[0] == x[2]:
                    x = node
                    break           
        solutionPath.append(x[0]) 
        solutionPath.reverse()
        
        #Code to generate searchPath
        while closed:
            searchPath.append(closed.pop()[0])    
        searchPath.reverse()
        timeTaken = 60-my_timer
        
        return solutionPath, searchPath, timeTaken
    else:
        return [-1], -1,-1

In [ ]:
# #Running test code:
# randomStart = [[6,1,2],[7,8,3],[5,4,9]]
# randomStart = [[1,2,3],[5,4,6],[7,9,8]]
# iterativeDeepening(randomStart)

In [ ]:
#Defining the heuristics for algorithm A*

#Hamming distance: h(n)<=h*(n) thus the heuristic is admissible 
def h1(state):
    i = 1
    h = 0
    for row in range(dimension):
        for col in range(dimension):
            if not state[row][col]== i:
                h += 1
            i += 1
    return h
            
#Permutation Inversion
def h2(state):
    h = 0
    list1 = [j for i in state for j in i]
    k = 0
    
    for row in range(dimension):
        if not row == 0:
            k += 2
        for col in range(dimension):
            #Taking the number tile
            temp = state[row][col]
            #iterating through all the values to the right of the tile
            for i in range((row+col+k),dimension*dimension):
                if (temp - list1[i]) > 0:
                    h += 1
        
    return h

In [ ]:
# # Test Code
# randomStart = [[6,1,2],[7,8,3],[5,4,9]]
# randomStart = [[1,2,3],[4,5,6],[7,9,8]]
# print(h2(randomStart))

In [ ]:
#Defining the algorithm A*

def sortByH(childTuple):
    return childTuple[2]

def algorithmA(initialState, h):
    success = False
    openPQ = []
    #Nodes are of form: [state,parent,h(n)]
    #Where [-1] will function as the root null
    openPQ.append([initialState,[-1],h(initialState)])
    closed = []
    counter = 0
    
    countdown_thread = threading.Thread(target = countdown)
    countdown_thread.start()
    while openPQ and (my_timer > 0):
        x = openPQ.pop(0)
        if checkGoal(x[0]):
            closed.append(x)
            success = True
            break
        else:
            children = getChildrenWithHeuristic(x[0],h)
            closed.append(x)
            #Check if children are already in the open or closed list
            for childTuple in children:
                if childTuple in openPQ or childTuple in closed:
                    children.remove(childTuple)
                else:
                    #Pushing valid children (left-end of open)
                    openPQ.append(childTuple)
            #Sorting        
            openPQ.sort(key=sortByH)
            sleep(1)  
            
            
    if success:
        estimatedCost = 0
        solutionPath = []
        searchPath = []
        
         #Code to genereate solution path   
        while not x[1] == [-1]:
            solutionPath.append(x[0])
            estimatedCost += x[2]
            for node in closed:
                if node[0] == x[1]:
                    x = node
                    break           
        solutionPath.append(x[0]) 
        solutionPath.reverse()
        
        #Code to generate searchPath
        while closed:
            searchPath.append(closed.pop()[0])
        searchPath.reverse()
        
        timeTaken = 60-my_timer
        
        return solutionPath, searchPath, estimatedCost,timeTaken
    
    else:
        return [-1],-1,-1,-1

In [993]:
# # Test Code
# randomStart = [[6,1,2],[7,8,3],[5,4,9]]
# # randomStart = [[1,2,3],[4,5,6],[7,9,8]]
# # randomStart = randomState()
# # print(randomStart)
# # randomStart = [[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,16,15]]
# solution, search, error, timeTaken = algorithmA(randomStart,h1)

# print("SEARCH PATH")
# for i in search:
#     print(i)
# print("SOLUTION PATH")
# for i in solution:
#     print(i)



SEARCH PATH
[[6, 1, 2], [7, 8, 3], [5, 4, 9]]
[[6, 1, 3], [7, 8, 2], [5, 4, 9]]
[[6, 1, 3], [5, 8, 2], [7, 4, 9]]
[[6, 1, 3], [5, 4, 2], [7, 8, 9]]
[[6, 1, 3], [4, 5, 2], [7, 8, 9]]
[[1, 6, 3], [4, 5, 2], [7, 8, 9]]
[[1, 5, 3], [4, 6, 2], [7, 8, 9]]
[[1, 6, 3], [4, 5, 2], [7, 8, 9]]
[[1, 5, 3], [4, 2, 6], [7, 8, 9]]
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
SOLUTION PATH
[[6, 1, 2], [7, 8, 3], [5, 4, 9]]
[[6, 1, 3], [7, 8, 2], [5, 4, 9]]
[[6, 1, 3], [5, 8, 2], [7, 4, 9]]
[[6, 1, 3], [5, 4, 2], [7, 8, 9]]
[[6, 1, 3], [4, 5, 2], [7, 8, 9]]
[[1, 6, 3], [4, 5, 2], [7, 8, 9]]
[[1, 5, 3], [4, 6, 2], [7, 8, 9]]
[[1, 5, 3], [4, 2, 6], [7, 8, 9]]
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


In [978]:
#Output file generation

#Random states generation 
import pickle 
def generateRandom():
    with open('20random.txt','wb') as fp:
        for i in range(20):
            randomStart = randomState()
            pickle.dump(randomStart, fp)
            
def readRandom():
    allPuzzles = []
    with open('20random.txt','rb') as fp:
        for i in range(20):
            allPuzzles.append(pickle.load(fp)) 
    return allPuzzles

    
# DFS output generation
def dfsToFile():
    searchFileCreated = False 
    solutionFileCreated = False
    allPuzzles = readRandom()
    puzzleNum = 0
    noSolution = 0
    totalExecutionTime = 0
    avgExecutionTime = 0
    totalSolutionLen = 0
    totalSearchLen = 0
    avgSolutionLen = 0
    avgSearchLen = 0
    
    for randomStart in allPuzzles:
        solution, search,executionTime = depthFirst(randomStart)
        if solution[0] == -1:
            noSolution += 1
        else:
            
            totalExecutionTime += executionTime
            totalSolutionLen += len(solution)
            totalSearchLen += len(search)
            
            if searchFileCreated:
                with open("[DFS]-[SEARCH].txt", "a") as f:
                    f.write("Puzzle #" + str(puzzleNum) + "\n")
                    f.write("--------------------------------\n")
                    for i in search:
                        f.write(str(i))
                        f.write('\n')
                    f.close() 
            else:
                searchFileCreated = True
                with open("[DFS]-[SEARCH].txt", "w+") as f:
                    f.write("Puzzle #" + str(puzzleNum) + "\n")
                    f.write("--------------------------------\n")
                    for i in search:
                        f.write(str(i))
                        f.write('\n')
                    f.close()    
             
            if solutionFileCreated:
                with open("[DFS]-[SOLUTION].txt", "a") as f:
                    f.write("Puzzle #" + str(puzzleNum) + "\n")
                    f.write("--------------------------------\n")
                    for i in search:
                        f.write(str(i))
                        f.write('\n')
                    f.close()
            else:
                solutionFileCreated = True
                with open("[DFS]-[SOLUTION].txt", "w+") as f:
                    f.write("Puzzle #" + str(puzzleNum) + "\n")
                    f.write("--------------------------------\n")
                    for i in solution:
                        f.write(str(i))
                        f.write('\n')
                    f.close()
                            
            puzzleNum += 1
    
    if noSolution == len(allPuzzles):
        avgExecutionTime = 0
        avgSolutionLen = 0
        avgSearchLen = 0

    else:    
        avgExecutionTime = totalExecutionTime/(len(allPuzzles) - noSolution)
        avgSolutionLen = totalSolutionLen/(len(allPuzzles) - noSolution)
        avgSearchLen = totalSearchLen/(len(allPuzzles) - noSolution)
    
    with open("[DFS]-[STATS].txt", "w+") as f:
        f.write("Average length of solution paths: " + str(avgSolutionLen) + "\n")
        f.write("Total length of solution paths: " + str(totalSolutionLen) + "\n")
        f.write("Average length of search path: " + str(avgSearchLen) + "\n")
        f.write("Total length of search paths: " + str(totalSearchLen) + "\n")
        f.write("Total number of no solution: " + str(noSolution) + "\n")
        f.write("Average execution time: " + str(avgExecutionTime) + "\n")
        f.write("Total execution time: " + str(noSolution) + "\n")
        f.close()
        


   
 #iterative deepening output generation
def idToFile():
    searchFileCreated = False 
    solutionFileCreated = False
    allPuzzles = readRandom()
    puzzleNum = 0
    noSolution = 0
    totalExecutionTime = 0
    avgExecutionTime = 0
    totalSolutionLen = 0
    totalSearchLen = 0
    avgSolutionLen = 0
    avgSearchLen = 0
    

    for randomStart in allPuzzles:
        solution, search,executionTime = iterativeDeepening(randomStart)
        print("Solution[0]: "+str(solution[0]))
        if solution[0] == -1:
            noSolution += 1
        else:
            totalExecutionTime += executionTime
            totalSolutionLen += len(solution)
            totalSearchLen += len(search)
            
            if searchFileCreated:
                with open("[ID]-[SEARCH].txt", "a") as f:
                    f.write("Puzzle #" + str(puzzleNum) + "\n")
                    f.write("--------------------------------\n")
                    for i in search:
                        f.write(str(i))
                        f.write('\n')
                    f.close() 
            else:
                searchFileCreated = True
                with open("[ID]-[SEARCH].txt", "w+") as f:
                    f.write("Puzzle #" + str(puzzleNum) + "\n")
                    f.write("--------------------------------\n")
                    for i in search:
                        f.write(str(i))
                        f.write('\n')
                    f.close()    
             
            if solutionFileCreated:
                with open("[ID]-[SOLUTION].txt", "a") as f:
                    f.write("Puzzle #" + str(puzzleNum) + "\n")
                    f.write("--------------------------------\n")
                    for i in search:
                        f.write(str(i))
                        f.write('\n')
                    f.close()
            else:
                solutionFileCreated = True
                with open("[ID]-[SOLUTION].txt", "w+") as f:
                    f.write("Puzzle #" + str(puzzleNum) + "\n")
                    f.write("--------------------------------\n")
                    for i in solution:
                        f.write(str(i))
                        f.write('\n')
                    f.close()        
            puzzleNum += 1
            
    if noSolution == len(allPuzzles):
        avgExecutionTime = 0
        avgSolutionLen = 0
        avgSearchLen = 0

    else:    
        avgExecutionTime = totalExecutionTime/(len(allPuzzles) - noSolution)
        avgSolutionLen = totalSolutionLen/(len(allPuzzles) - noSolution)
        avgSearchLen = totalSearchLen/(len(allPuzzles) - noSolution)
    
    with open("[ID]-[STATS].txt", "w+") as f:
        f.write("Average length of solution paths: " + str(avgSolutionLen) + "\n")
        f.write("Total length of solution paths: " + str(totalSolutionLen) + "\n")
        f.write("Average length of search path: " + str(avgSearchLen) + "\n")
        f.write("Total length of search paths: " + str(totalSearchLen) + "\n")
        f.write("Total number of no solution: " + str(noSolution) + "\n")
        f.write("Average execution time: " + str(avgExecutionTime) + "\n")
        f.write("Total execution time: " + str(noSolution) + "\n")
        f.close()

#Algorithm A* h1 - hamming distance
def aH1():
    searchFileCreated = False 
    solutionFileCreated = False
    allPuzzles = readRandom()
    puzzleNum = 0
    noSolution = 0
    totalExecutionTime = 0
    avgExecutionTime = 0
    totalSolutionLen = 0
    totalSearchLen = 0
    avgSolutionLen = 0
    avgSearchLen = 0
    

    for randomStart in allPuzzles:
        solution, search,estimatedCost,executionTime = algorithmA(randomStart,h1)
        if solution[0] == -1:
            noSolution += 1
        else:
            totalExecutionTime += executionTime
            totalSolutionLen += len(solution)
            totalSearchLen += len(search)
            
            if searchFileCreated:
                with open("[AH1]-[SEARCH].txt", "a") as f:
                    f.write("Puzzle #" + str(puzzleNum) + "\n")
                    f.write("--------------------------------\n")
                    for i in search:
                        f.write(str(i))
                        f.write('\n')
                    f.close() 
            else:
                searchFileCreated = True
                with open("[AH1]-[SEARCH].txt", "w+") as f:
                    f.write("Puzzle #" + str(puzzleNum) + "\n")
                    f.write("--------------------------------\n")
                    for i in search:
                        f.write(str(i))
                        f.write('\n')
                    f.close()    
             
            if solutionFileCreated:
                with open("[AH1]-[SOLUTION].txt", "a") as f:
                    f.write("Puzzle #" + str(puzzleNum) + "\n")
                    f.write("--------------------------------\n")
                    for i in search:
                        f.write(str(i))
                        f.write('\n')
                    f.close()
            else:
                solutionFileCreated = True
                with open("[AH1]-[SOLUTION].txt", "w+") as f:
                    f.write("Puzzle #" + str(puzzleNum) + "\n")
                    f.write("--------------------------------\n")
                    for i in solution:
                        f.write(str(i))
                        f.write('\n')
                    f.close()
                            
            puzzleNum += 1
            
    if noSolution == len(allPuzzles):
        avgExecutionTime = 0
        avgSolutionLen = 0
        avgSearchLen = 0

    else:    
        avgExecutionTime = totalExecutionTime/(len(allPuzzles) - noSolution)
        avgSolutionLen = totalSolutionLen/(len(allPuzzles) - noSolution)
        avgSearchLen = totalSearchLen/(len(allPuzzles) - noSolution)

    with open("[AH1]-[STATS].txt", "w+") as f:
        f.write("Average length of solution paths: " + str(avgSolutionLen) + "\n")
        f.write("Total length of solution paths: " + str(totalSolutionLen) + "\n")
        f.write("Average length of search path: " + str(avgSearchLen) + "\n")
        f.write("Total length of search paths: " + str(totalSearchLen) + "\n")
        f.write("Total number of no solution: " + str(noSolution) + "\n")
        f.write("Average execution time: " + str(avgExecutionTime) + "\n")
        f.write("Total execution time: " + str(totalExecutionTime) + "\n")
        f.close()
    
    
# #Algorithm A* h2 - permutation Inversion
def aH2():
    searchFileCreated = False 
    solutionFileCreated = False
    allPuzzles = readRandom()
    puzzleNum = 0
    noSolution = 0
    totalExecutionTime = 0
    avgExecutionTime = 0
    totalSolutionLen = 0
    totalSearchLen = 0
    avgSolutionLen = 0
    avgSearchLen = 0
    

    for randomStart in allPuzzles:
        solution, search,estimatedCost,executionTime = algorithmA(randomStart,h2)
        if solution[0] == -1:
            noSolution += 1
        else:
            totalExecutionTime += executionTime
            totalSolutionLen += len(solution)
            totalSearchLen += len(search)
            
            if searchFileCreated:
                with open("[AH2]-[SEARCH].txt", "a") as f:
                    f.write("Puzzle #" + str(puzzleNum) + "\n")
                    f.write("--------------------------------\n")
                    for i in search:
                        f.write(str(i))
                        f.write('\n')
                    f.close() 
            else:
                searchFileCreated = True
                with open("[AH2]-[SEARCH].txt", "w+") as f:
                    f.write("Puzzle #" + str(puzzleNum) + "\n")
                    f.write("--------------------------------\n")
                    for i in search:
                        f.write(str(i))
                        f.write('\n')
                    f.close()    
             
            if solutionFileCreated:
                with open("[AH2]-[SOLUTION].txt", "a") as f:
                    f.write("Puzzle #" + str(puzzleNum) + "\n")
                    f.write("--------------------------------\n")
                    for i in search:
                        f.write(str(i))
                        f.write('\n')
                    f.close()
            else:
                solutionFileCreated = True
                with open("[AH2]-[SOLUTION].txt", "w+") as f:
                    f.write("Puzzle #" + str(puzzleNum) + "\n")
                    f.write("--------------------------------\n")
                    for i in solution:
                        f.write(str(i))
                        f.write('\n')
                    f.close()
                            
            puzzleNum += 1
    
    if noSolution == len(allPuzzles):
        avgExecutionTime = 0
        avgSolutionLen = 0
        avgSearchLen = 0

    else:    
        avgExecutionTime = totalExecutionTime/(len(allPuzzles) - noSolution)
        avgSolutionLen = totalSolutionLen/(len(allPuzzles) - noSolution)
        avgSearchLen = totalSearchLen/(len(allPuzzles) - noSolution)
    
    with open("[AH2]-[STATS].txt", "w+") as f:
        f.write("Average length of solution paths: " + str(avgSolutionLen) + "\n")
        f.write("Total length of solution paths: " + str(totalSolutionLen) + "\n")
        f.write("Average length of search path: " + str(avgSearchLen) + "\n")
        f.write("Total length of search paths: " + str(totalSearchLen) + "\n")
        f.write("Total number of no solution: " + str(noSolution) + "\n")
        f.write("Average execution time: " + str(avgExecutionTime) + "\n")
        f.write("Total execution time: " + str(totalExecutionTime) + "\n")
        f.close()

In [994]:
# generateRandom()
# Code to print to file all results
# dfsToFile()
# idToFile()
# aH1()
# aH2()


In [ ]:
#Change dimension, and call this algo
aH2()
